<a href="https://colab.research.google.com/github/Shirouky/lab2-Data-analysis/blob/main/LR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [273]:
import requests

n = 40

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}


for i in range(n):
  url = f"https://biser-optom.ru/category/biser-Preciosa?page={i}"
  response = requests.get(url, headers=headers, timeout=10)

  if response.status_code == 200:
    with open(f"beads{i}.html", 'w') as f:
        f.write(response.text)
  else:
    print(f"Ошибка: {response.status_code}")

#Задание 2

In [274]:
def decorator(func):
  def inner(*args):
    try:
      return func(*args)
    except:
      return None
  return inner

In [275]:
def digit(func):
  def inner(*args):
    result = func(*args)

    if isinstance(result, str):
      temp = result.split()
      for t in temp:
        try:
          return int(t)
        except ValueError:
          continue

    return result
  return inner

In [282]:
class Card():
  image = ""
  procent = 0
  currency = ""
  price = 0
  old_price = 0
  quantity = 0
  code = ""
  id_web = 0
  id = 0
  url = ""
  type_ = ""
  size = ""
  weight = ""
  color = ""

  def __init__(self, card):
    self.image = self.set_image(card)
    self.procent = self.set_procent(card)
    self.currency = self.set_currency(card)
    self.price = self.set_price(card)
    self.old_price = self.set_old_price(card)
    self.quantity = self.set_quantity(card)
    self.code = self.set_code(card)
    self.id_web = self.set_id_web(card)
    self.url = self.set_url(card)

    self.set_name(card)

  @decorator
  def set_name(self, card):
    name = card.find("div", "goods-image").find("img").get("alt")
    temp = name.split()
    if (len(name) < 6):
      name = {}
    else:
      self.type_ = temp[2]
      self.size = temp[3]
      self.weight = self.set_weight(temp[4][:-1])
      self.color = temp[6:]

    return name

  @digit
  def set_weight(self, data):
    return data

  @decorator
  def set_image(self, card):
    return card.find("div", "goods-image").find('a').get("href")

  @digit
  @decorator
  def set_procent(self, card):
    return card.find("span", "badge-sales").text

  @decorator
  def set_currency(self, card):
    return card.find("meta").get("content")

  @digit
  @decorator
  def set_price(self, card):
    return card.find("span", itemprop="price").text

  @digit
  @decorator
  def set_old_price(self, card):
    return card.find("span", "old_price").text

  @digit
  @decorator
  def set_quantity(self, card):
    return card.find("div", "availability").find("span").text

  @decorator
  def set_code(self, card):
    return card.find("div", itemprop="description").find("span").text

  @digit
  @decorator
  def set_id_web(self, card):
    return card.find("input", autocomplete="off").get("data-id")

  def set_id(self, id):
    self.id = id

  @decorator
  def set_url(self, card):
    return card.find("meta", itemprop="url").get("content")

  def export(self):
    data = {}
    data["id"] = self.id
    data["id_web"] = self.id_web
    data["code"] = self.code
    data["type"] = self.type_
    data["size"] = self.size
    data["weight"] = self.weight
    data["color"] = self.color
    data["procent"] = self.procent
    data["price"] = self.price
    data["old_price"] = self.old_price
    data["currency"] = self.currency
    data["quantity"] = self.quantity
    data["image"] = self.image
    data["url"] = self.url
    return data

In [283]:
from bs4 import BeautifulSoup

cards = []
j = 0

for i in range(n):
  with open (f"beads{i}.html") as answer_t:
    answer = answer_t.read()

  soup_page = BeautifulSoup(answer, "html.parser")
  div_titles = soup_page.find_all("div", "card")[:-1]

  for line in div_titles:
    j += 1
    card = Card(line)
    card.set_id(j)
    cards.append(card.export())
line

<div class="card" itemscope="" itemtype="http://schema.org/Product"> <div class="goods-image"> <a href="/uploads/goods//jV6iHBd5jr0cxEc.jpg" rel="facebox"> <img alt="Бисер Чехия круглый 10/0 500г 18181 золотой" class="lazy" data-src="/uploads/goods//c200/thumnail_jV6iHBd5jr0cxEc.jpg" itemprop="image"/> </a> <div style="bottom: -6px; left: 20px; position: absolute;"><span class="badge badge-pill badge-success badge-sales">-12 %</span> </div> <div class="favorite-absolute"> <div class="favorite-wrapper"> <div> <a class="add-to-fa-no-login" data-id="169446" href="" title="В Избранное"> <i class="fa fa-heart-o"></i> </a> </div> </div> </div> </div> <div class="card-body"> <div class="gallery__item__price" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer"> <div class="row price-modifiers"> </div> <meta content="RUB" itemprop="priceCurrency"/> <meta content="https://biser-optom.ru/goods/2562" itemprop="url"/> <link href="http://schema.org/InStock" itemprop="availability"/> <d

In [278]:
from json import dump

def json_save(output_data):
    with open("beads.json", 'w') as f:
        dump(output_data, f, indent=2, ensure_ascii=False)

    print("Данные сохранены в json")

json_save(cards)

Данные сохранены в json
